# A Light Introduction to the Internals of Python
AI Meetup 22.05.2020

## What I won't cover in this talk:
- lexer (source code -> tokens)
- parser (tokens -> ast)
- grammar (Python has a very easy grammar)


## Agenda
- the cpython repository
- compilation
- bytecode
- the dis module
- opcodes
- the ceval.c loop and Python virtual machine
- what does it mean that in Python everything is an object

### We will be talking about Cpython

## CPython repository https://github.com/python/cpython
- easy to compile and setup for development
- most important directories: 
  - Include: header files
  - Objects: object implementations, from int to type
  - Python: interpreter, bytecode compiler and other essential infrastructure
- other:
  - Modules: stdlib extension modules
  - Lib: Python standard library implemented in Python


## High level overview of running a Python script
\*.py -> compiler -> bytecode -> main iterpreter loop

## Compilation
In CPython, the compilation from source code to bytecode involves several steps:

1. Parse source code into a parse tree (Parser/pgen.c)
2. Transform parse tree into an Abstract Syntax Tree (Python/ast.c)
3. Transform AST into a Control Flow Graph (Python/compile.c)
4. Emit bytecode based on the Control Flow Graph (Python/compile.c)

## Let's start with a simple Python script

In [1]:
script = """
G = 1

class A:
    def __init__(self, func):
        self.func = func

def func(n):
    return G + n

"""

In [2]:
from ast import parse
ast_ = parse(script)

In [3]:
ast_.body

In [4]:
ast_.body[2].body[0].value

In [5]:
ast_.body[1].body[0].name

'__init__'

In [6]:
co = compile(ast_, 'test.py', 'exec')

In [7]:
{code:getattr(co, code) for code in dir(co) if not code.startswith('__')}

{'co_argcount': 0,
 'co_cellvars': (),
 'co_code': b'd\x00Z\x00G\x00d\x01d\x02\x84\x00d\x02\x83\x02Z\x01d\x03d\x04\x84\x00Z\x02d\x05S\x00',
 'co_consts': (1,
  <code object A at 0x7f5e86e34240, file "test.py", line 4>,
  'A',
  <code object func at 0x7f5e86e342f0, file "test.py", line 8>,
  'func',
  None),
 'co_filename': 'test.py',
 'co_firstlineno': 2,
 'co_flags': 8256,
 'co_freevars': (),
 'co_kwonlyargcount': 0,
 'co_lnotab': b'\x04\x02\x0e\x04',
 'co_name': '<module>',
 'co_names': ('G', 'A', 'func'),
 'co_nlocals': 0,
 'co_posonlyargcount': 0,
 'co_stacksize': 3,
 'co_varnames': (),
 'replace': <function code.replace(*, co_argcount=-1, co_posonlyargcount=-1, co_kwonlyargcount=-1, co_nlocals=-1, co_stacksize=-1, co_flags=-1, co_firstlineno=-1, co_code=None, co_consts=None, co_names=None, co_varnames=None, co_freevars=None, co_cellvars=None, co_filename=None, co_name=None, co_lnotab=None)>}

In [8]:
co.co_filename

'test.py'

In [9]:
co.co_code

b'd\x00Z\x00G\x00d\x01d\x02\x84\x00d\x02\x83\x02Z\x01d\x03d\x04\x84\x00Z\x02d\x05S\x00'

In [10]:
[x for x in co.co_code]

[100,
 0,
 90,
 0,
 71,
 0,
 100,
 1,
 100,
 2,
 132,
 0,
 100,
 2,
 131,
 2,
 90,
 1,
 100,
 3,
 100,
 4,
 132,
 0,
 90,
 2,
 100,
 5,
 83,
 0]

We can see what every opcode does in the docs https://docs.python.org/3/library/dis.html

- defined in `Include/opcode.h`
- opcode 90

In [11]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [12]:
import dis

In [13]:
dis.dis(co)

  2           0 LOAD_CONST               0 (1)
              2 STORE_NAME               0 (G)

  4           4 LOAD_BUILD_CLASS
              6 LOAD_CONST               1 (<code object A at 0x7f5e86e34240, file "test.py", line 4>)
              8 LOAD_CONST               2 ('A')
             10 MAKE_FUNCTION            0
             12 LOAD_CONST               2 ('A')
             14 CALL_FUNCTION            2
             16 STORE_NAME               1 (A)

  8          18 LOAD_CONST               3 (<code object func at 0x7f5e86e342f0, file "test.py", line 8>)
             20 LOAD_CONST               4 ('func')
             22 MAKE_FUNCTION            0
             24 STORE_NAME               2 (func)
             26 LOAD_CONST               5 (None)
             28 RETURN_VALUE

Disassembly of <code object A at 0x7f5e86e34240, file "test.py", line 4>:
  4           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_C

## Maybe we should start with something easier

In [14]:
def f():
    ...

In [15]:
dis.dis(f)

  2           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE


In [16]:
def f():
    x = 1
    return x

In [17]:
dis.dis(f)

  2           0 LOAD_CONST               1 (1)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
              6 RETURN_VALUE


In [18]:
class A:
        ...

In [19]:
dis.dis(A)

In [20]:
def _():
    class A:
        ...

In [21]:
dis.dis(_)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object A at 0x7f5e86dd6ea0, file "<ipython-input-20-a5730c675798>", line 2>)
              4 LOAD_CONST               2 ('A')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('A')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (A)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE

Disassembly of <code object A at 0x7f5e86dd6ea0, file "<ipython-input-20-a5730c675798>", line 2>:
  2           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('_.<locals>.A')
              6 STORE_NAME               2 (__qualname__)

  3           8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


## Fun fact: some statements of the Python grammar have no corresponding executable bytecode -- they have only effect in compilation time

In [22]:
def _(x=1):
    def f():
        return x

In [23]:
dis.dis(_)

  2           0 LOAD_CLOSURE             0 (x)
              2 BUILD_TUPLE              1
              4 LOAD_CONST               1 (<code object f at 0x7f5e86dd6a80, file "<ipython-input-22-14deac161694>", line 2>)
              6 LOAD_CONST               2 ('_.<locals>.f')
              8 MAKE_FUNCTION            8 (closure)
             10 STORE_FAST               1 (f)
             12 LOAD_CONST               0 (None)
             14 RETURN_VALUE

Disassembly of <code object f at 0x7f5e86dd6a80, file "<ipython-input-22-14deac161694>", line 2>:
  3           0 LOAD_DEREF               0 (x)
              2 RETURN_VALUE


In [24]:
G = 1
def _():
    return G

In [25]:
dis.dis(_)

  3           0 LOAD_GLOBAL              0 (G)
              2 RETURN_VALUE


In [26]:
x = 1
def _():
    x += 1  # the same as x = x + 1

In [27]:
_()

UnboundLocalError: local variable 'x' referenced before assignment

In [28]:
dis.dis(_)

  3           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_FAST               0 (x)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [29]:
x = 1
def _():
    global x
    x += 1

In [30]:
_()
x

2

In [31]:
dis.dis(_)

  4           0 LOAD_GLOBAL              0 (x)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (x)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [32]:
def _():
    x = 1
    def f():
        nonlocal x
        x += 1

In [33]:
dis.dis(_)

  2           0 LOAD_CONST               1 (1)
              2 STORE_DEREF              0 (x)

  3           4 LOAD_CLOSURE             0 (x)
              6 BUILD_TUPLE              1
              8 LOAD_CONST               2 (<code object f at 0x7f5e86e34190, file "<ipython-input-32-77d10288b2dd>", line 3>)
             10 LOAD_CONST               3 ('_.<locals>.f')
             12 MAKE_FUNCTION            8 (closure)
             14 STORE_FAST               0 (f)
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE

Disassembly of <code object f at 0x7f5e86e34190, file "<ipython-input-32-77d10288b2dd>", line 3>:
  5           0 LOAD_DEREF               0 (x)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_DEREF              0 (x)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [34]:
def f():
    locals()['m'] = 1
    print(locals()['m'])    
    print(f'm = {m}')


In [35]:
f()

1


NameError: name 'm' is not defined

In [36]:
dis.dis(f)

  2           0 LOAD_CONST               1 (1)
              2 LOAD_GLOBAL              0 (locals)
              4 CALL_FUNCTION            0
              6 LOAD_CONST               2 ('m')
              8 STORE_SUBSCR

  3          10 LOAD_GLOBAL              1 (print)
             12 LOAD_GLOBAL              0 (locals)
             14 CALL_FUNCTION            0
             16 LOAD_CONST               2 ('m')
             18 BINARY_SUBSCR
             20 CALL_FUNCTION            1
             22 POP_TOP

  4          24 LOAD_GLOBAL              1 (print)
             26 LOAD_CONST               3 ('m = ')
             28 LOAD_GLOBAL              2 (m)
             30 FORMAT_VALUE             0
             32 BUILD_STRING             2
             34 CALL_FUNCTION            1
             36 POP_TOP
             38 LOAD_CONST               0 (None)
             40 RETURN_VALUE


In [37]:
def _(x,y):
    return x + y

In [38]:
dis.dis(_)

  2           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 RETURN_VALUE


In [39]:
def _(x, y):
    return x.__add__(y)

In [40]:
dis.dis(_)

  2           0 LOAD_FAST                0 (x)
              2 LOAD_METHOD              0 (__add__)
              4 LOAD_FAST                1 (y)
              6 CALL_METHOD              1
              8 RETURN_VALUE


Python doesn't know what `x` and `y` is. Python just tries it at runtime

## Main interpreter loop vs. performance

A common misconception is that `__add__` is the same as the `+` operator (etc.)

In [41]:
x = 1

In [42]:
%timeit x + x

33.7 ns ± 0.656 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [43]:
%timeit x.__add__(x)

83.9 ns ± 2.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [44]:
%timeit '1' + '1'

6.57 ns ± 0.192 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [45]:
%timeit '1'.__add__('1')

106 ns ± 3.52 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Why do we see such difference??



## DEMO TIME!!!
Goto Python/ceval.c

In [46]:
StackFrames, every frame has 3 stacks,  every function has a separate frame, PyObjectVar, structural subtyping

SyntaxError: invalid syntax (<ipython-input-46-033d5ec356b0>, line 1)